# Set up

In [5]:
import os
from ollama import Client

In [6]:
# LLM_SERVER_HOST = '192.168.100.16'
LLM_SERVER_HOST = '192.168.100.10'
LLM_SERVER_PORT = 11434
client = Client(host=f'http://{LLM_SERVER_HOST}:{LLM_SERVER_PORT}')

# Test prompt

In [27]:
system_prompt = """
You are a helpful expert Machine Learning annotator.

Given these text reviews, your task is to generate structured training data for a Named Entity Recognition (NER) task.

Requirements:
- Entities are defined as the phrase or clause that contains a full statement mentioning a reviewed aspect.
- List of useful aspects: FOOD, BEVERAGE, AMBIENCE, SERVICE, LOCATION, MUSIC, KITCHEN, PRICE, VIEW, GOOD_FOR
- The extracted phrases/clauses should be the shortest possible while still containing enough information about the sentiment regarding the aspect.
- The extracted phrases/clauses should be a subset of the input text.
- The extracted phrases/clauses can be overlapping with each other.

Output should strictly follow the JSON format of a dictionary where each key is the id of the input element and each value is the corresponding extracted data. Each value should follow this schema:
- text (string): the original text
- entities (list of lists): the extracted entities, where each entity contains:
  + First element: the sub text
  + Second element: the annotated aspect
  + Confidence score (float): range from 0.01 to 0.99, denotes how confident you are about the annotations
  + Sentiment (float): range from -1.0 to 1.0, where -1.0 is very negative and +1.0 is very positive

Example 1:
Input:
[
  {"id": "1", "text": "But the staff was so horrible to us."},
  {"id": "2", "text": "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not."},
  {"id": "3", "text": "I particularly love their yellowfin tuna and their mussel selection."},
  {"id": "4", "text": "The ambiance of the restaurant was very soothing and calming."},
  {"id": "5", "text": "Their wine selection is impressive, offering a variety of rare bottles."},
  {"id": "6", "text": "I found the prices to be a bit on the higher side."},
  {"id": "7", "text": "The restaurant offers a stunning view of the city skyline."},
  {"id": "8", "text": "It's a great place for family dinners."},
  {"id": "9", "text": "I didn't enjoy the music, it was too loud."},
  {"id": "10", "text": "The restaurant's location is very convenient, close to the subway."}
]

Output:
{
  "1": {
    "text": "But the staff was so horrible to us.",
    "entities": [
      ["staff was so horrible", "SERVICE", 0.99, -0.9]
    ]
  },
  "2": {
    "text": "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.",
    "entities": [
      ["food is uniformly exceptional", "FOOD", 0.99, 0.9],
      ["very capable kitchen", "KITCHEN", 0.99, 0.8]
    ]
  },
  "3": {
    "text": "I particularly love their yellowfin tuna and their mussel selection.",
    "entities": [
      ["love their yellowfin tuna", "FOOD", 0.9, 0.8],
      ["love their yellowfin tuna and their mussel selection", "FOOD", 0.8, 0.7]
    ]
  },
  "4": {
    "text": "The ambiance of the restaurant was very soothing and calming.",
    "entities": [
      ["ambiance of the restaurant was very soothing", "AMBIENCE", 0.95, 0.85]
    ]
  },
  "5": {
    "text": "Their wine selection is impressive, offering a variety of rare bottles.",
    "entities": [
      ["wine selection is impressive", "BEVERAGE", 0.95, 0.8]
    ]
  },
  "6": {
    "text": "I found the prices to be a bit on the higher side.",
    "entities": [
      ["prices to be a bit on the higher side", "PRICE", 0.9, -0.4]
    ]
  },
  "7": {
    "text": "The restaurant offers a stunning view of the city skyline.",
    "entities": [
      ["stunning view of the city skyline", "VIEW", 0.98, 0.9]
    ]
  },
  "8": {
    "text": "It's a great place for family dinners.",
    "entities": [
      ["great place for family dinners", "GOOD_FOR", 0.95, 0.85]
    ]
  },
  "9": {
    "text": "I didn't enjoy the music, it was too loud.",
    "entities": [
      ["didn't enjoy the music", "MUSIC", 0.85, -0.7]
    ]
  },
  "10": {
    "text": "The restaurant's location is very convenient, close to the subway.",
    "entities": [
      ["location is very convenient", "LOCATION", 0.92, 0.7]
    ]
  }
}

---
Example 2:
Input:
[
  {"id": "1", "text": "The service was quick and friendly."},
  {"id": "2", "text": "I loved the elegant interior design of the restaurant."},
  {"id": "3", "text": "The pasta dishes are absolutely delicious and well worth the price."},
  {"id": "4", "text": "They have an extensive cocktail menu with unique creations."},
  {"id": "5", "text": "It's a perfect spot for romantic dates."},
  {"id": "6", "text": "The live music added a great touch to the evening."},
  {"id": "7", "text": "Their desserts are to die for."},
  {"id": "8", "text": "The restaurant is located in a busy part of town, making it easily accessible."},
  {"id": "9", "text": "The kitchen was slow in preparing our meals."},
  {"id": "10", "text": "We enjoyed the panoramic view from the rooftop terrace."}
]

Output:
{
  "1": {
    "text": "The service was quick and friendly.",
    "entities": [
      ["service was quick and friendly", "SERVICE", 0.98, 0.9]
    ]
  },
  "2": {
    "text": "I loved the elegant interior design of the restaurant.",
    "entities": [
      ["loved the elegant interior design", "AMBIENCE", 0.9, 0.8]
    ]
  },
  "3": {
    "text": "The pasta dishes are absolutely delicious and well worth the price.",
    "entities": [
      ["pasta dishes are absolutely delicious", "FOOD", 0.95, 0.85],
      ["well worth the price", "PRICE", 0.85, 0.7]
    ]
  },
  "4": {
    "text": "They have an extensive cocktail menu with unique creations.",
    "entities": [
      ["extensive cocktail menu", "BEVERAGE", 0.9, 0.75]
    ]
  },
  "5": {
    "text": "It's a perfect spot for romantic dates.",
    "entities": [
      ["perfect spot for romantic dates", "GOOD_FOR", 0.9, 0.85]
    ]
  },
  "6": {
    "text": "The live music added a great touch to the evening.",
    "entities": [
      ["live music added a great touch", "MUSIC", 0.88, 0.8]
    ]
  },
  "7": {
    "text": "Their desserts are to die for.",
    "entities": [
      ["desserts are to die for", "FOOD", 0.95, 0.9]
    ]
  },
  "8": {
    "text": "The restaurant is located in a busy part of town, making it easily accessible.",
    "entities": [
      ["located in a busy part of town", "LOCATION", 0.9, 0.7],
      ["easily accessible", "LOCATION", 0.85, 0.6]
    ]
  },
  "9": {
    "text": "The kitchen was slow in preparing our meals.",
    "entities": [
      ["kitchen was slow in preparing", "KITCHEN", 0.8, -0.6]
    ]
  },
  "10": {
    "text": "We enjoyed the panoramic view from the rooftop terrace.",
    "entities": [
      ["panoramic view from the rooftop terrace", "VIEW", 0.9, 0.85]
    ]
  }
}

---
Example 3:
Input:
[
  {"id": "1", "text": "The seating arrangement could be better."},
  {"id": "2", "text": "I felt the lighting was a bit too dim for my liking."},
  {"id": "3", "text": "The waiting time was not great but the food was decent."},
  {"id": "4", "text": "The decor is quite old-fashioned and could use some updates."},
  {"id": "5", "text": "I think the desserts were okay, not the best."},
  {"id": "6", "text": "The noise level was quite high, making it hard to have a conversation."},
  {"id": "7", "text": "I wasn't too impressed with the salad bar, but the drinks were fine."},
  {"id": "8", "text": "The restrooms were clean, but the hand dryers didn't work well."},
  {"id": "9", "text": "The view from the window wasn't as good as I expected."},
  {"id": "10", "text": "The ambiance is neither good nor bad, just average."}
]

Output:
{
  "1": {
    "text": "The seating arrangement could be better.",
    "entities": [
      ["seating arrangement could be better", "AMBIENCE", 0.6, -0.3]
    ]
  },
  "2": {
    "text": "I felt the lighting was a bit too dim for my liking.",
    "entities": [
      ["lighting was a bit too dim", "AMBIENCE", 0.5, -0.4]
    ]
  },
  "3": {
    "text": "The waiting time was not great but the food was decent.",
    "entities": [
      ["waiting time was not great", "SERVICE", 0.5, -0.5],
      ["food was decent", "FOOD", 0.7, 0.3]
    ]
  },
  "4": {
    "text": "The decor is quite old-fashioned and could use some updates.",
    "entities": [
      ["decor is quite old-fashioned", "AMBIENCE", 0.4, -0.4],
      ["could use some updates", "AMBIENCE", 0.6, -0.2]
    ]
  },
  "5": {
    "text": "I think the desserts were okay, not the best.",
    "entities": [
      ["desserts were okay", "FOOD", 0.6, 0.1],
      ["not the best", "FOOD", 0.5, -0.3]
    ]
  },
  "6": {
    "text": "The noise level was quite high, making it hard to have a conversation.",
    "entities": [
      ["noise level was quite high", "AMBIENCE", 0.6, -0.6]
    ]
  },
  "7": {
    "text": "I wasn't too impressed with the salad bar, but the drinks were fine.",
    "entities": [
      ["wasn't too impressed with the salad bar", "FOOD", 0.6, -0.5],
      ["drinks were fine", "BEVERAGE", 0.7, 0.3]
    ]
  },
  "8": {
    "text": "The restrooms were clean, but the hand dryers didn't work well.",
    "entities": [
      ["restrooms were clean", "SERVICE", 0.7, 0.4],
      ["hand dryers didn't work well", "SERVICE", 0.5, -0.5]
    ]
  },
  "9": {
    "text": "The view from the window wasn't as good as I expected.",
    "entities": [
      ["view from the window wasn't as good", "VIEW", 0.4, -0.4]
    ]
  },
  "10": {
    "text": "The ambiance is neither good nor bad, just average.",
    "entities": [
      ["ambiance is neither good nor bad", "AMBIENCE", 0.3, 0.0]
    ]
  }
}

Important!:
- Do not include any notes in your output to keep the output is a valid JSON string.
- Output should begin with `[` and end with `]`
"""

In [28]:
prompt = """
Input:
{input_texts}
"""

In [29]:
APPROX_CHARS_PER_TOKEN = 4
prompt_approx_token_counts = (len(system_prompt) + len(prompt)) / APPROX_CHARS_PER_TOKEN
print(f"{prompt_approx_token_counts=}")

prompt_approx_token_counts=2444.25


In [31]:
%%time
# input_texts = [
#   "But the staff was so horrible to us.",
#   "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.",
#   "this little place has a cute interior decor and affordable city prices."
# ]
input_texts = [
    {"id": "1", "text": "They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it."},
    {"id": "2", "text": "The pizza is the best if you like thin crusted pizza."},
    {"id": "3", "text": "All the money went into the interior decoration, none of it went to the chefs."}
]
input_texts_formatted = [{"text": text} for text in input_texts]

response = client.chat(
    model='llama3',
    messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": prompt.format(input_texts=str(input_texts_formatted))
        }
    ],
    options=dict(
        temperature=0,
        num_predict=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    ),
    format='json'
)
print(response['message']['content'])

{
  "1": {
    "text": "They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it.",
    "entities": [
      ["did not have mayonnaise", "FOOD", 0.5, -0.6],
      ["forgot our toast", "SERVICE", 0.4, -0.7],
      ["left out ingredients (ie cheese in an omelet)", "FOOD", 0.6, -0.5],
      ["below hot temperatures", "SERVICE", 0.3, -0.8],
      ["bacon was so over cooked it crumbled on the plate when you touched it", "FOOD", 0.2, -0.9]
    ]
  },
  "2": {
    "text": "The pizza is the best if you like thin crusted pizza.",
    "entities": [
      ["pizza is the best", "FOOD", 0.8, 0.6],
      ["like thin crusted pizza", "PREFERENCE", 0.5, 0.3]
    ]
  },
  "3": {
    "text": "All the money went into the interior decoration, none of it went to the chefs.",
    "entities": [
      ["all the money went into the interior decoration", "AMBIENCE", 0.4, -0.6]

# Archive